In [7]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from scipy.special import logit
from scipy.stats import norm
from functions import  feature_engineering, train_test_split, one_hot
import warnings
warnings.filterwarnings("ignore")

In [8]:
data = pd.read_csv("/Users/victorialokteva/Downloads/data.csv")
data = data.drop(columns=["oaid_hash", 'impressions', 'campaign_clicks'])

In [12]:
data = feature_engineering(data)


KeyboardInterrupt: 

In [ ]:
X_train, Y_train, X_test0, X_test1, Y_test, coefs = train_test_split(data)
X_train, X_test0, X_test1 = one_hot(X_train), one_hot(X_test0), one_hot(X_test1 )

In [ ]:
# Модель из 1 дз
model = LogisticRegression(penalty='l2', solver='liblinear', C=0.1)
model.fit(X_train, Y_train)

In [ ]:
y_pred0 = model.predict(X_test0)
y_pred1 = model.predict(X_test1)
coefs['coeff_sum0_new'] = logit(model.predict_proba(X_test0)[:, 1])
coefs['coeff_sum1_new'] = logit(model.predict_proba(X_test1)[:, 1])

In [ ]:
coefs['pi_0'] = norm.sf((coefs['coeff_sum1'] -
                          coefs['coeff_sum0']) / np.sqrt(coefs['g0'] ** 2 + coefs['g1'] ** 2))
coefs['pi_1'] = norm.sf((coefs['coeff_sum1_new'] -
                          coefs['coeff_sum0_new']) / np.sqrt(coefs['g0'] ** 2 + coefs['g1'] ** 2))

In [ ]:

lam = 10
cips = np.sum(Y_test * np.clip(coefs['pi_1'] / coefs['pi_0'], a_min=None, a_max=lam)) / len(Y_test)

print(cips)